# Import

In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2 torchaudio==0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install thai2transformers
!pip install transformers
!pip install pythainlp
!pip install sentencepiece

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.4MB/s eta 0:03:45tcmalloc: large alloc 1147494400 bytes == 0x555970f1e000 @  0x7f6ba6db0615 0x555936c5d06c 0x555936d3ceba 0x555936c5fe8d 0x555936d5199d 0x555936cd3fe9 0x555936cceb0e 0x555936c6177a 0x555936cd3e50 0x555936cceb0e 0x555936c6177a 0x555936cd086a 0x555936d527c6 0x555936ccfee2 0x555936d527c6 0x555936ccfee2 0x555936d527c6 0x555936ccfee2 0x555936d527c6 0x555936dd4431 0x555936d35049 0x555936c9fc84 0x555936c608e9 0x555936cd4ade 0x555936c6169a 0x555936ccfa45 0x555936ccee0d 0x555936c6177a 0x555936ccfa45 0x555936c6169a 0x555936ccfa45
     |█████████████████████████████▏  | 1055.7MB 1.4MB/s eta 0:01:13tcmalloc: large alloc 1434370048 bytes == 0x5559b5574000 @  0x7f6ba6db0615 0x555936c5d06c 0x555936d3ceba 0x555936c5fe8d 0x555936d5199d 0x555936cd3fe9 0x555936cceb0e 0x555936c6177a 0x555936cd3e50 0x555936cceb0e 0x555936c6177a 0x555936cd086a 0x555936d527c6 0x555936ccfee2 0

In [ ]:
!gdown --id 1Avz8kLB8Ko1Gn2TPMY-Zz214grt-gOa7

Downloading...
From: https://drive.google.com/uc?id=1Avz8kLB8Ko1Gn2TPMY-Zz214grt-gOa7
To: /content/thailand_tweet.csv
11.8MB [00:00, 44.9MB/s]


In [ ]:
import pandas as pd
import numpy as np
import torch
import math
import pickle
import random
import multiprocessing as mp
import regex as re

from sklearn.model_selection import train_test_split

import torch
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, pipeline

import thai2transformers
from thai2transformers.preprocess import process_transformers

In [ ]:
torch.__version__

'1.7.1+cu110'

In [ ]:
RNG_SEED = 0
pl.seed_everything(RNG_SEED)

Global seed set to 0


0

# Preprocessing

In [ ]:
thai_tweets = pd.read_csv('thailand_tweet.csv')
thai_tweets.head(10)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions
0,1.196659e+18,tGpRofwHLXI5iuye+ZDKVW8yVCic9mkRLxTwiBbEOrU=,tGpRofwHLXI5iuye+ZDKVW8yVCic9mkRLxTwiBbEOrU=,tGpRofwHLXI5iuye+ZDKVW8yVCic9mkRLxTwiBbEOrU=,NaN,NaN,NaN,0,3,2019-11-05,en,th,กกต.จี้“อนาคตใหม่”แจงเงินกู้“ธนาธร” https://t....,2019-11-19 5:20,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,1,[],['https://www.thansettakij.com/content/politic...,[]
1,1.197369e+18,tGpRofwHLXI5iuye+ZDKVW8yVCic9mkRLxTwiBbEOrU=,tGpRofwHLXI5iuye+ZDKVW8yVCic9mkRLxTwiBbEOrU=,tGpRofwHLXI5iuye+ZDKVW8yVCic9mkRLxTwiBbEOrU=,NaN,NaN,NaN,0,3,2019-11-05,en,th,ตายสนิท!'อิศรา'เปิดภ.ง.ด.91ปี61'คุณช่อ'แจ้งราย...,2019-11-21 4:20,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],['https://www.thaipost.net/main/detail/49119?f...,[]
2,1.195602e+18,hfg2zB3w3Ms2yTAnnRWD3XtVmgsIhmP+FKfELGctzU=,hfg2zB3w3Ms2yTAnnRWD3XtVmgsIhmP+FKfELGctzU=,hfg2zB3w3Ms2yTAnnRWD3XtVmgsIhmP+FKfELGctzU=,NaN,ก็ไม่มีรัยมากนะ,NaN,0,3,2019-11-05,th,und,Let's Get It : ชีวิตติดใบแดง CD กันต์ธีร์ EP....,2019-11-16 7:19,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,1,0,0,0,[],['https://youtu.be/3PL1_B6hGt4'],['10228272']
3,1.204279e+18,lktnIU8UKhYmk56OS2jEPjVTNidDUUnlrlDQCMRsPUA=,lktnIU8UKhYmk56OS2jEPjVTNidDUUnlrlDQCMRsPUA=,lktnIU8UKhYmk56OS2jEPjVTNidDUUnlrlDQCMRsPUA=,NaN,NaN,NaN,17,38,2019-10-01,th,th,อ้าวเฮ้ย ไม่เหมือนที่คุยกันไว้นี่หว่า...อุตส่า...,2019-12-10 5:58,Twitter Web App,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],['https://m.youtube.com/watch?v=zTOfftM62fQ&fe...,[]
4,1.227887e+18,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,NaN,NaN,NaN,5,42,2020-01-14,th,th,เป้นกำลังใจให้นะท่าน,2020-02-13 9:26,Twitter Web App,NaN,NaN,1.226497e+18,False,NaN,NaN,absent,absent,0,0,0,0,[],[],[]
5,1.229678e+18,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,NaN,NaN,NaN,5,42,2020-01-14,th,th,ไอ้หน้าเหี้ย,2020-02-18 8:05,Twitter Web App,NaN,NaN,1.229613e+18,False,NaN,NaN,absent,absent,0,0,0,0,[],[],[]
6,1.227148e+18,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,AY56F7lcfIPCx1p6y5EFqbfw3CfIBpQLhW6oVmcI7og=,NaN,NaN,NaN,5,42,2020-01-14,th,th,อย่าทำให้มันเป้นเรื่องเศร้า,2020-02-11 8:29,Twitter Web App,NaN,NaN,1.226860e+18,False,NaN,NaN,absent,absent,0,0,0,0,[],[],[]
7,1.204282e+18,5D5m7kUmuyEs+yq6qxazy5gz2evjQ40wJQRnfTP4Ayk=,5D5m7kUmuyEs+yq6qxazy5gz2evjQ40wJQRnfTP4Ayk=,5D5m7kUmuyEs+yq6qxazy5gz2evjQ40wJQRnfTP4Ayk=,NaN,NaN,NaN,3,9,2019-12-06,en,und,https://t.co/62ynfXcgkI,2019-12-10 6:09,Twitter for Android,NaN,NaN,1.203978e+18,False,NaN,NaN,absent,absent,0,0,0,0,[],['https://twitter.com/hengsuaycountry/status/1...,[]
8,1.203537e+18,6ENbhsd2AhxafBPn0WJSkDxk3Mr19Ua9dAlyHrOQmU4=,6ENbhsd2AhxafBPn0WJSkDxk3Mr19Ua9dAlyHrOQmU4=,6ENbhsd2AhxafBPn0WJSkDxk3Mr19Ua9dAlyHrOQmU4=,NaN,NaN,NaN,4,9,2019-11-05,en,th,คนทำก็ไม่รู้ใช่อะไรคิด อาจารย์ก็ไม่รู้ยอมได้ไง...,2019-12-08 4:47,Twitter Web App,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],['https://m.facebook.com/story.php?story_fbid=...,[]
9,1.227794e+18,zWXEXafRx2edxmywiQFD38NKzFMaTzGzjQO1IgMisgs=,zWXEXafRx2edxmywiQFD38NKzFMaTzGzjQO1IgMisgs=,zWXEXafRx2edxmywiQFD38NKzFMaTzGzjQO1IgMisgs=,NaN,เป็นคนดีค่ะ,NaN,0,4,2019-12-06,en,th,RT @MaisonWanvipa: ปี 2562 ขาดประชุม ส.ส.ไป 1 ...,2020-02-13 3:19,Twitter for iPhone,NaN,NaN,NaN,True,NaN,1.224613e+18,absent,absent,0,0,0,0,"['ทํางานวนไป', 'อนาคตใหม่']",[],['1082182785501020160']


In [ ]:
thai_tweets['is_retweet'].value_counts()

True     11199
False    10186
Name: is_retweet, dtype: int64

# Clean text

In [ ]:
def remove_retweets(df):
  return df[df.is_retweet == False]

In [ ]:
URL_TOKEN = '<link>'
#https://stackoverflow.com/questions/27745/getting-parts-of-a-url-regex
URL_EXTRACTION_REGEX = r'\b((http[s]?|ftp):\/)?\/?([^:\/\s]+)((\/\w+)*\/)([\w\-\.]+[^#?\s]+)(.*)?(#[\w\-]+)?\b'

def translate_message_urls(message_content):
    return re.sub(URL_EXTRACTION_REGEX, URL_TOKEN, message_content, flags=re.IGNORECASE)

def remove_message_urls(message_content):
    return re.sub(URL_EXTRACTION_REGEX, '', message_content, flags=re.IGNORECASE)

def map_message_urls(df, remove=False):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_message_urls if remove else translate_message_urls)
    return new_df

In [ ]:
MENTION_TOKEN = '<mention>'
#https://stackoverflow.com/questions/8650007/regular-expression-for-twitter-username
# TWITTER_USERNAME_REGEX = r'(^|[^@\w])@(\w{1,15})\b'
TWITTER_USERNAME_REGEX = r'@[a-zA-Z0-9_./^!-]*'

def translate_mentions(message_content):
    return re.sub(TWITTER_USERNAME_REGEX, MENTION_TOKEN, message_content, flags=re.IGNORECASE)

def remove_mentions(message_content):
    return re.sub(TWITTER_USERNAME_REGEX, '', message_content, flags=re.IGNORECASE)

def map_mentions(df, remove=False):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_mentions if remove else translate_mentions)
    return new_df

In [ ]:
def fix_misspelings(message_content):
    message_content = re.sub('ยังงัย', 'ยังไง', message_content)
    message_content = re.sub(r'ก้อ|ก้', 'ก็', message_content)
    return message_content

def map_misspelings(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(fix_misspelings)
    return new_df

In [ ]:
def remove_hashtags(message_content):
  return re.sub('#', '', message_content)

def map_hashtags(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_hashtags)
    return new_df

In [ ]:
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
EMOJI_PATTERN = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u200b"
                      "]+", re.UNICODE)

def remove_emojis(message_content):
  return re.sub(EMOJI_PATTERN, '', message_content)

def map_emojis(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_emojis)
    return new_df

In [ ]:
def translate_saraae(message_content):
  return re.sub('เเ', 'แ', message_content)

def map_saraae(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(translate_saraae)
    return new_df

In [ ]:
def remove_linefeed(message_content):
  return re.sub('\n', ' ', message_content)

def map_linefeed(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_linefeed)
    return new_df

In [ ]:
# Edited from: https://stackoverflow.com/questions/1660694/regular-expression-to-match-any-character-being-repeated-more-than-10-times
REPEATED_DOT_REGEX = r'([.])\1{2,}'

def remove_repeated_dots(message_content):
  return re.sub(REPEATED_DOT_REGEX, ' ', message_content)

def map_repeated_dots(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_repeated_dots)
    return new_df

In [ ]:
LAUGH_REGEX = r'([5])\1{2,}'

def remove_laughs(message_content):
  return re.sub(LAUGH_REGEX, '', message_content)

def map_laughs(df):
    new_df = df.copy()
    new_df['tweet_text'] = df['tweet_text'].apply(remove_laughs)
    return new_df

In [ ]:
REPEATED_KRUB_REGEX = r'([ครับ][บ])\1{1,}'

def remove_krub(message_content):
  message_content = re.sub(REPEATED_KRUB_REGEX, '', message_content)
  return re.sub('ครับ', '', message_content)

def map_krub(df):
  new_df = df.copy()
  new_df['tweet_text'] = df['tweet_text'].apply(remove_krub)
  return new_df

In [ ]:
def strip_df(df):
  new_df = df.copy()
  new_df['tweet_text'] = df['tweet_text'].str.strip()
  return new_df

def remove_empty(df):
  new_df = df.copy()
  new_df = df[df['tweet_text'].str.len() > 0]
  return new_df

In [ ]:
"""
TODO:
- Remove \n (possibly the reason of idx 10 token prediction)
- Tone down ครับ?​ Or do something so the model doesn't always try to output it
- Remove continuous punctuations (e.g. '...')
"""

cleaned_tweets = thai_tweets.copy()
cleaned_tweets = remove_retweets(cleaned_tweets)
cleaned_tweets = map_message_urls(cleaned_tweets, remove=True)
cleaned_tweets = map_mentions(cleaned_tweets, remove=True)
cleaned_tweets = map_misspelings(cleaned_tweets)
# cleaned_tweets = map_hashtags(cleaned_tweets)
cleaned_tweets = map_emojis(cleaned_tweets)
cleaned_tweets = map_saraae(cleaned_tweets)
cleaned_tweets = map_linefeed(cleaned_tweets)
cleaned_tweets = map_repeated_dots(cleaned_tweets)
cleaned_tweets = map_laughs(cleaned_tweets)
# cleaned_tweets = map_krub(cleaned_tweets)
cleaned_tweets = strip_df(cleaned_tweets)
cleaned_tweets = remove_empty(cleaned_tweets)

In [ ]:
def inspect_tweet_text(df, size, return_df=False):
  start = int(random.random() * df.shape[0])
  stop = min(df.shape[0], start + size)
  if return_df: return df.iloc[start:stop,:]
  return df.iloc[start:stop,:]['tweet_text'].values

In [ ]:
inspect_tweet_text(cleaned_tweets, 30)

array(['ดูหน้าก็รู้ว่าตอแหลมากเลย',
       'ดูถูกไทยด้วยกันเอง ยกยอต่างชาติเป็นแบบอย่าง',
       'แล้วม๊อบจะจัดการยังไง',
       'ว่าแต่เขาอิเหนาเป็นเองจริงๆ ทั้งเรื่องบ้านเรื่องที่ดิน',
       'ใช่คะท่านลงไปเพื่อช่วยเหลือนะคะ', 'เยี่ยมเลยคะ',
       'เยี่ยมมากขอเป็นกำลังใจไห้คะ', 'ขอพระองค์ทรงพระเจริญ',
       'มันจะได้ผลหรือคะ', 'มันเป็นเรื่องของบุคคนเขานะ',
       'ไม่มีผลอะไรหลอกคะ',
       'ในความเป็น "ทหาร" ก็เสียใจกับเหตุการณ์นี้ #ไม่เคยโทษทหาร #ทหารไม่เคยสอนให้ฆ่าผู้บริสุทธิ์ #strongkorat',
       'เป็นกำลังใจให้ทุกคนครับ', '#กลัวที่ไหน #ธนาธร',
       'ทำไมไวรัสโคโรน่ามันน่ากลัวจังครับ',
       '::เปิดแล้ว!!! “สายตรงแก็ไขปัญหาสำหรับกำลังพลกองทัพบก 02-018-7330 ขอให้กำลังพล ยืนยันว่าเป็นกำลังพลของ ทบ. ทุกเรื่องถูกส่งตรงถึง ผบ.ทบ. และเป็นความลับที่สุด  รายละเอียดเพิ่มเติม   #สายตรงแก็ไขปัญหาสำหรับกำลังพลกองทัพบก #Hotlineสายตรงทบ.  #RTA call center',
       'รอดูคะจะสู้ไหวไหมคะ', 'ดูแลตัวเองด้วยนะ', 'ขอไห้ป้องกันไห้ดีคะ',
       'ดูไปก่อนนะคะ',
       'เอะอะก็ฟ้อง   ค้านทุกเรื่องย

In [ ]:
import string

s = '::เปิดแล้ว!!! “สายตรง'

s.strip(string.punctuation)

'เปิดแล้ว!!! “สายตรง'

In [ ]:
VAL_RATIO = 0.15
TEST_RATIO = 0.10

df_train, df_test = train_test_split(cleaned_tweets, test_size=VAL_RATIO+TEST_RATIO)
df_val, df_test = train_test_split(df_test, test_size=TEST_RATIO/(VAL_RATIO+TEST_RATIO))

In [ ]:
def get_text(df):
  return list(df['tweet_text'])

tweets_train = get_text(df_train)
tweets_val = get_text(df_val)
tweets_test = get_text(df_test)

In [ ]:
inspect_tweet_text(df_test, 10)

array(['อย่าทำให้มันเป้นเรื่องใหย่เลยนะ', 'เลอะเทอะจังพวกคุณเนี้ยยย ย ย',
       'ประเพณีและวัฒนธรรมของคนไทย ต้องอนุรักษ์และรักษาไว้ให้คงอยู่สืบต่อไป  จะอายทำไมในเมื่อเราเป็นชนชาติที่มีวัฒนธรรมเป็นของตัวเอง  ที่คุณถามว่าจะหมายถึงการแสดงอำนาจจะเป็นไปได้อย่างไร  ในเมื่อเป็นพิธีประเพณีโบราณ  ที่ใช้ประกอบขบวนต่างๆก็เป็นอาวุธโบราณตามราชประเพณี',
       'ยุบไปมันก็ไม่หนุกนะสิการเมืองไทย', 'ขอไห้ป้องกันควบคุมไห้ได้คะ',
       'เหตุการณ์นี้ทาง ผบ.ทบ.เองก็ไม่ได้โทษใครนี่ ทางกองทัพเองก็รับผิดชอบต่อผู้ที่ประสบเหตุ',
       'ระวังน่ะครับ', 'เหรอ', 'ใส่ใจกันด้วยนะครับเพื่อตัวคุณเอง',
       'ดูแล้วเรียกยังไงก็คงไม่ไป'], dtype=object)

# Dataset

In [ ]:
MODEL_NAME = 'airesearch/wangchanberta-base-att-spm-uncased'
SHOULD_ADD_SPEC_TOKEN = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if SHOULD_ADD_SPEC_TOKEN:
  added_tokens_count = tokenizer.add_tokens([URL_TOKEN, MENTION_TOKEN], special_tokens=True)
else:
  try:
    del added_tokens_count
  except:
    pass

In [ ]:
MAX_LENGTH = 256

def tokenize_message(message):
  return tokenizer(
    message,
    truncation=True,
    padding='max_length',
    max_length=MAX_LENGTH
  )

def tokenize_entry(message):
  encoded_message = tokenize_message(message)
  input_ids = np.array(encoded_message['input_ids'])
  attention_mask = np.array(encoded_message['attention_mask'])
  return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [ ]:
EXCLUDED_TOKEN_IDS = [tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.cls_token_id]
IGNORED_TOKEN_ID = -100

MASKED_ODDS = 0.15
NEW_TOKEN_ODDS = 0.10
OLD_TOKEN_ODDS = 0.10

In [ ]:
def mask_tokens(input_ids):
  vf = np.vectorize(mask_token)
  return vf(input_ids)

def mask_token(input_id):
  if is_masked(input_id):
    new_token_id = generate_masked_token_id(input_id)
    return (new_token_id, input_id)
  return (input_id, IGNORED_TOKEN_ID)

def is_masked(input_id):
  if input_id in EXCLUDED_TOKEN_IDS:
    return False
  if np.random.random_sample() <= MASKED_ODDS:
    return True
  return False

def generate_masked_token_id(input_id):
  odd = np.random.random_sample()
  if odd <= NEW_TOKEN_ODDS:
    return generate_new_token_id(tokenizer.vocab_size)
  if odd <= NEW_TOKEN_ODDS + OLD_TOKEN_ODDS:
    return input_id
  return tokenizer.mask_token_id

def generate_new_token_id(vocab_size):
  new_token = np.random.randint(0, vocab_size)
  while new_token in EXCLUDED_TOKEN_IDS:
    new_token = np.random.randint(0, vocab_size)
  return new_token

In [ ]:
class BertIODataset(torch.utils.data.Dataset):
  def __init__(self, dataset):
    self.dataset = dataset

  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    entry = self.dataset[idx]
    result = tokenize_entry(entry)
    new_input_ids, labels = mask_tokens(result['input_ids'])
    attention_mask = result['attention_mask']

    input_ids = torch.from_numpy(new_input_ids)
    labels = torch.from_numpy(labels)
    attention_mask = torch.from_numpy(attention_mask)

    return { 
      'input_ids': input_ids, 
      'labels' : labels, 
      'attention_mask' : attention_mask
    }

In [ ]:
BATCH_SIZE = 16

enc_tweets_train = BertIODataset(tweets_train)
enc_tweets_val = BertIODataset(tweets_val)
enc_tweets_test = BertIODataset(tweets_test)

train_loader = DataLoader(enc_tweets_train, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)
val_loader = DataLoader(enc_tweets_val, batch_size=BATCH_SIZE, pin_memory=True)
test_loader = DataLoader(enc_tweets_test, batch_size=BATCH_SIZE, pin_memory=True)

In [ ]:
tokenizer.decode(enc_tweets_test[100]['input_ids'])

'<s><mask>เทคโนโลยี<mask>ประโยชน์ครับ</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
enc_tweets_test

# BERT has a mouth

In [ ]:
!pip install nltk

In [ ]:
from transformers import (AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

In [ ]:
model_version = 'airesearch/wangchanberta-base-att-spm-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_version, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_version)

In [ ]:
model.load_state_dict(torch.load('wc_io_w_ht.pth'))

<All keys matched successfully>

In [ ]:
# model.bert.eval()
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    # model.bert = model.bert.cuda()
    model = model.cuda()

# Load pre-trained model tokenizer (vocabulary)
# tokenizer = AutoTokenizer.from_pretrained(model_version)

def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '<s>'
SEP = '</s>'
MASK = '<mask>'

mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

In [ ]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    # out = out['logits']
    out = out[0]
    logits = out[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx
  
  
def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    batch = [seed_text + [MASK] * max_len + [SEP] for _ in range(batch_size)]
    #if rand_init:
    #    for ii in range(max_len):
    #        init_idx[seed_len+ii] = np.random.randint(0, len(tokenizer.vocab))
    
    return tokenize_batch(batch)

def post_processing(sentence):
    ret = sentence.replace('<_>', " ")
    ret = ret.replace('▁', " ")
    ret = ret.replace('_', " ")
    ret = ret.replace('<s>', '')
    ret = ret.replace('</s>', '')
    return ret.strip(' ')

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)
    print("".join(sent))

def returner(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)
    ret = "".join(sent)
    ret = post_processing(ret)
    return (ret)

This is the meat of the algorithm. The general idea is
1. start from all masks
2. repeatedly pick a location, mask the token at that location, and generate from the probability distribution given by BERT
3. stop when converged or tired of waiting

We consider three "modes" of generating:
- generate a single token for a position chosen uniformly at random for a chosen number of time steps
- generate in sequential order (L->R), one token at a time
- generate for all positions at once for a chosen number of time steps

The `generate` function wraps and batches these three generation modes. In practice, we find that the first leads to the most fluent samples.

In [ ]:
# Generation modes as functions
import math
import time

def parallel_sequential_generation(seed_text, batch_size=10, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = tokenizer.convert_ids_to_tokens(batch[0])
            for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("iter", ii+1, " ".join(for_print))
            
    return untokenize_batch(batch)

def parallel_generation(seed_text, batch_size=10, max_len=15, top_k=0, temperature=None, max_iter=300, sample=True, 
                        cuda=False, print_every=10, verbose=True):
    """ Generate for all positions at each time step """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        for kk in range(max_len):
            idxs = generate_step(out, gen_idx=seed_len+kk, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii, print_every) == 0:
            print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(batch[0])))
    
    return untokenize_batch(batch)
            
def sequential_generation(seed_text, batch_size=10, max_len=15, leed_out_len=15, 
                          top_k=0, temperature=None, sample=True, cuda=False):
    """ Generate one word at a time, in L->R order """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_len):
        inp = [sent[:seed_len+ii+leed_out_len]+[sep_id] for sent in batch]
        print(inp)
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        idxs = generate_step(out, gen_idx=seed_len+ii, top_k=top_k, temperature=temperature, sample=sample)
        for jj in range(batch_size):
            batch[jj][seed_len+ii] = idxs[jj]
        
    return untokenize_batch(batch)


def generate(n_samples, seed_text="<s>", batch_size=10, max_len=25, 
             generation_mode="parallel-sequential",
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    # main generation function to call
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    print(n_batches)
    start_time = time.time()
    for batch_n in range(n_batches):
        if generation_mode == "parallel-sequential":
            batch = parallel_sequential_generation(seed_text, batch_size=batch_size, max_len=max_len, top_k=top_k,
                                                   temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                                   cuda=cuda, verbose=False)
        elif generation_mode == "sequential":
            batch = sequential_generation(seed_text, batch_size=batch_size, max_len=max_len, top_k=top_k, 
                                          temperature=temperature, leed_out_len=leed_out_len, sample=sample,
                                          cuda=cuda)
        elif generation_mode == "parallel":
            batch = parallel_generation(seed_text, batch_size=batch_size,
                                        max_len=max_len, top_k=top_k, temperature=temperature, 
                                        sample=sample, max_iter=max_iter, 
                                        cuda=cuda, verbose=False)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        
        sentences += batch
    return sentences

Let's call the actual generation function! We'll use the following settings
- max_len (40): length of sequence to generate
- top_k (100): at each step, sample from the top_k most likely words
- temperature (1.0): smoothing parameter for the next word distribution. Higher means more like uniform; lower means more peaky
- burnin (250): for non-sequential generation, for the first burnin steps, sample from the entire next word distribution, instead of top_k
- max_iter (500): number of iterations to run for
- seed_text (["CLS"]): prefix to generate for. We found it crucial to start with the CLS token; you can try adding to it 

In [ ]:
n_samples = 5
batch_size = 5
max_len = 15
top_k = 100
temperature = .8
generation_mode = "parallel-sequential"
leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 500

def gen_sent(seed_text, max_len=max_len):
  seed_text = tokenizer.tokenize(seed_text)
  print(f'seed text: {seed_text}')
  bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                      generation_mode=generation_mode,
                      sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                      cuda=cuda)
  for sent in bert_sents:
    printer(sent, should_detokenize=False)

def gen_sent_from_tok(seed_text, max_len=max_len):
  # seed_text = tokenizer.tokenize(seed_text)
  print(f'seed text: {seed_text}')
  bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                      generation_mode=generation_mode,
                      sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                      cuda=cuda)
  sents = []
  for sent in bert_sents:
    sents.append(returner(sent, should_detokenize=False))
  
  return sents
  
# Choose the prefix context
# seed_text = tokenizer.tokenize("<s>พรรคอนาคตใหม่")
# print(f'seed text: {seed_text}')
# bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
#                       generation_mode=generation_mode,
#                       sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
#                       cuda=cuda)

## Evaluation

In [ ]:
testto = []
seed_len = 4

for sent in tweets_test:
  tok = tokenizer.tokenize('<s>'+sent)
  if len(tok) > seed_len:
    testto.append(tok)

First 100  sentence

In [ ]:
# first 100
refs = []
cans = []
for sent_tok in testto[:100]:
  seed_text = sent_tok[:seed_len]
  gen_len = len(sent_tok) - seed_len

  ref = [sent_tok+['</s>']]
  can = gen_sent_from_tok(seed_text, gen_len)
  can = [['<s>']+tokenizer.tokenize(x)+['</s>'] for x in can]
  m_idx = 0
  m_bleu = -1
  for i,e in enumerate(can):
    b_score = sentence_bleu(ref, e)
    if b_score > m_bleu:
      m_bleu = b_score
      m_idx = i
  
  candidate = can[m_idx]
  
  refs.append(ref)
  cans.append(candidate)

  # print(tok)

seed text: ['<s>', '▁ทําให้', 'มีประสิทธิ', 'ภาพ']
1
Finished batch 1 in 11.180s
seed text: ['<s>', '▁พวก', 'ส', 'ถุน']
1
Finished batch 1 in 10.889s
seed text: ['<s>', '▁', 'เคยเจอ', 'คนที่เคย']
1
Finished batch 1 in 10.980s
seed text: ['<s>', '▁คง', 'ทํามา', 'กําจัด']
1
Finished batch 1 in 11.150s
seed text: ['<s>', '▁ต้อง', 'อยุ่', 'แบบ']
1
Finished batch 1 in 10.895s
seed text: ['<s>', '▁', 'สนับสนุน', '▁']
1
Finished batch 1 in 10.982s
seed text: ['<s>', '▁', 'เป็น', 'กําลังใจ']
1
Finished batch 1 in 11.109s
seed text: ['<s>', '▁', 'เปลี่ยนจาก', 'กองทัพบก']
1
Finished batch 1 in 15.107s
seed text: ['<s>', '▁', 'เป็น', 'บู']
1
Finished batch 1 in 11.077s
seed text: ['<s>', '▁เรา', 'ป', 'วงชนชาวไทย']
1
Finished batch 1 in 14.317s
seed text: ['<s>', '▁', 'เป็น', 'กําลังใจ']
1
Finished batch 1 in 10.880s
seed text: ['<s>', '▁', 'ยอด', 'เย']
1
Finished batch 1 in 10.901s
seed text: ['<s>', '▁', '+', 'WAX']
1
Finished batch 1 in 15.564s
seed text: ['<s>', '▁เพื่อเป็น', 'การ', 'ให้เกียรต

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Finished batch 1 in 14.797s
seed text: ['<s>', '▁ขอให้', 'จัดระเบียบ', 'ให้']
1
Finished batch 1 in 11.053s
seed text: ['<s>', '▁ผมจะ', 'หมด', 'หัว']
1
Finished batch 1 in 11.078s
seed text: ['<s>', '▁', 'ว่า', 'กันไป']
1
Finished batch 1 in 10.951s
seed text: ['<s>', '▁', 'เย', 'ื่']
1
Finished batch 1 in 11.176s
seed text: ['<s>', '▁อย่า', 'ทําให้มัน', 'เป็นเรื่อง']
1
Finished batch 1 in 11.178s
seed text: ['<s>', '▁เลิก', 'ทําให้มัน', 'ว']
1
Finished batch 1 in 10.854s
seed text: ['<s>', '▁กู', 'ว่า', 'นะ']
1
Finished batch 1 in 14.472s
seed text: ['<s>', '▁', 'ยินดีกับ', 'ท่าน']
1
Finished batch 1 in 11.004s
seed text: ['<s>', '▁', 'ตาย', 'ฟรี']
1
Finished batch 1 in 15.245s
seed text: ['<s>', '▁', '#', 'กองทัพภาค']
1
Finished batch 1 in 15.015s
seed text: ['<s>', '▁ความจริง', 'เป็น', 'แบบไหน']
1
Finished batch 1 in 10.943s
seed text: ['<s>', '▁', 'ขอไห้', 'เรา']
1
Finished batch 1 in 14.524s
seed text: ['<s>', '▁', '#', 'กองทัพบก']
1
Finished batch 1 in 11.140s
seed text: ['<s>', 

In [ ]:
# first 100 
corpus_bleu(refs, cans)

0.2967104671899176

Sentence 100 - 300 

In [ ]:
# 100 - 300 
refs2 = []
cans2 = []

for sent_tok in testto[100:300]:
  seed_text = sent_tok[:seed_len]
  gen_len = len(sent_tok) - seed_len

  ref = [sent_tok+['</s>']]
  can = gen_sent_from_tok(seed_text, gen_len)
  can = [['<s>']+tokenizer.tokenize(x)+['</s>'] for x in can]
  m_idx = 0
  m_bleu = -1
  for i,e in enumerate(can):
    b_score = sentence_bleu(ref, e)
    if b_score > m_bleu:
      m_bleu = b_score
      m_idx = i
  
  candidate = can[m_idx]
  
  refs2.append(ref)
  cans2.append(candidate)

seed text: ['<s>', '▁ทหาร', 'ช่วยเหลือ', 'ประชาชน']
1
Finished batch 1 in 11.376s
seed text: ['<s>', '▁นัก', 'การเมือง', 'อย่า']
1
Finished batch 1 in 15.256s
seed text: ['<s>', '▁เห็นด้วย', 'กับเขา', 'นะ']
1
Finished batch 1 in 14.816s
seed text: ['<s>', '▁', 'บ้าไปแล้ว', '▁']
1
Finished batch 1 in 11.298s
seed text: ['<s>', '▁เรื่องนี้', 'ทําเป็น', 'เมิน']
1
Finished batch 1 in 27.116s
seed text: ['<s>', '▁', 'ท', 'รงพระเจริญ']
1
Finished batch 1 in 10.998s
seed text: ['<s>', '▁', 'ว่า', 'กันไป']
1
Finished batch 1 in 20.370s
seed text: ['<s>', '▁ต้อง', 'แบบนี้แหละ', 'ครับ']
1
Finished batch 1 in 15.184s
seed text: ['<s>', '▁เอา', 'ใ', 'ง']
1
Finished batch 1 in 11.176s
seed text: ['<s>', '▁ขอให้', 'คนที่', 'สร้าง']
1
Finished batch 1 in 14.517s
seed text: ['<s>', '▁', 'ขอแสดงความ', 'เสียใจกับ']
1
Finished batch 1 in 11.087s
seed text: ['<s>', '▁', 'สวด', 'ด']
1
Finished batch 1 in 11.356s
seed text: ['<s>', '▁', 'เป็น', 'พรรค']
1
Finished batch 1 in 15.023s
seed text: ['<s>', '▁ขอให

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Finished batch 1 in 11.159s
seed text: ['<s>', '▁', 'นี่แหล่ะ', 'ที่เขา']
1
Finished batch 1 in 21.779s
seed text: ['<s>', '▁', 'น้อม', 'กราบ']
1
Finished batch 1 in 11.216s
seed text: ['<s>', '▁', 'ทําดี', 'ต่อไป']
1
Finished batch 1 in 11.156s
seed text: ['<s>', '▁ก็', 'ว่า', 'กันต่อไป']
1
Finished batch 1 in 11.020s
seed text: ['<s>', '▁', 'จะออกมา', 'ทิศทาง']
1
Finished batch 1 in 11.030s
seed text: ['<s>', '▁มันเป็น', 'คนทํางาน', 'ที่']
1
Finished batch 1 in 10.976s
seed text: ['<s>', '▁', 'สู้ต่อไป', 'ครับ']
1
Finished batch 1 in 11.434s
seed text: ['<s>', '▁เลิก', 'ทําให้', 'บ้านเมือง']
1
Finished batch 1 in 11.265s
seed text: ['<s>', '▁', 'เป็น', 'กําลังใจ']
1
Finished batch 1 in 11.244s
seed text: ['<s>', '▁ใช่', 'ๆ', 'ถูกต้อง']
1
Finished batch 1 in 11.290s
seed text: ['<s>', '▁', 'มีความ', 'คิดเห็น']
1
Finished batch 1 in 11.026s
seed text: ['<s>', '▁', 'เป็น', 'กําลังใจ']
1
Finished batch 1 in 14.512s
seed text: ['<s>', '▁พร้อมทั้ง', 'กาย', 'ทั้ง']
1
Finished batch 1 in 11.

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Finished batch 1 in 11.295s
seed text: ['<s>', '▁ใคร', 'ผิด', 'ก็']
1
Finished batch 1 in 11.029s
seed text: ['<s>', '▁ช่วย', 'ๆ', 'กัน']
1
Finished batch 1 in 11.249s
seed text: ['<s>', '▁', 'ทบ', '.']
1
Finished batch 1 in 28.411s
seed text: ['<s>', '▁ท่าน', 'สุดยอด', 'ครับ']
1
Finished batch 1 in 11.139s
seed text: ['<s>', '▁รอดู', 'แนวทาง', 'ของพรรค']
1
Finished batch 1 in 11.194s
seed text: ['<s>', '▁', 'เป็นนัก', 'กฏหมาย']
1
Finished batch 1 in 11.097s
seed text: ['<s>', '▁', '#', 'ขอ']
1
Finished batch 1 in 15.209s
seed text: ['<s>', '▁ขอบคุณ', 'กองทัพ', 'นะครับ']
1
Finished batch 1 in 11.002s
seed text: ['<s>', '▁', 'ภาพที่', 'เต็มไปด้วย']
1
Finished batch 1 in 11.088s
seed text: ['<s>', '▁', 'เป็น', 'กําลังใจ']
1
Finished batch 1 in 11.154s
seed text: ['<s>', '▁เวลา', 'พวก', 'ม']
1
Finished batch 1 in 10.981s
seed text: ['<s>', '▁ขอบคุณ', 'จิตอาสา', 'ทุกๆคน']
1
Finished batch 1 in 11.324s
seed text: ['<s>', '▁', 'อยู่', 'พรรคนี้']
1
Finished batch 1 in 10.934s
seed text: ['<s>

In [ ]:
# first 300
corpus_bleu(refs+refs2, cans+cans2)

0.31703706854564606

Sentence 300 - 500

In [ ]:
# 300 - 500 
for sent_tok in testto[300:500]:
  seed_text = sent_tok[:seed_len]
  gen_len = len(sent_tok) - seed_len

  ref = [sent_tok+['</s>']]
  can = gen_sent_from_tok(seed_text, gen_len)
  can = [['<s>']+tokenizer.tokenize(x)+['</s>'] for x in can]
  m_idx = 0
  m_bleu = -1
  for i,e in enumerate(can):
    b_score = sentence_bleu(ref, e)
    if b_score > m_bleu:
      m_bleu = b_score
      m_idx = i
  
  candidate = can[m_idx]
  
  refs2.append(ref)
  cans2.append(candidate)

seed text: ['<s>', '▁อย่า', 'ใช้คําว่า', 'ประชาชน']
1
Finished batch 1 in 15.317s
seed text: ['<s>', '▁ศาล', 'รัฐธรรมนูญ', 'ยุติธรรม']
1
Finished batch 1 in 11.149s
seed text: ['<s>', '▁อย่า', 'โทษ', 'เกม']
1
Finished batch 1 in 11.073s
seed text: ['<s>', '▁อี', 'หยัง', 'ของ']
1
Finished batch 1 in 15.639s
seed text: ['<s>', '▁', 'กองทัพ', 'ไม่มีวัน']
1
Finished batch 1 in 11.058s
seed text: ['<s>', '▁เอา', 'เวลา', 'ตั้ง']
1
Finished batch 1 in 11.027s
seed text: ['<s>', '▁ใช่', 'ๆ', 'ธร']
1
Finished batch 1 in 11.266s
seed text: ['<s>', '▁', 'เค้าก็', 'กําลัง']
1
Finished batch 1 in 11.046s
seed text: ['<s>', '▁', 'ไอเดีย', 'ดีมาก']
1
Finished batch 1 in 14.310s
seed text: ['<s>', '▁', 'สร้างความ', 'เดือดร้อน']
1


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Finished batch 1 in 15.104s
seed text: ['<s>', '▁', 'อนาคตใหม่', 'ทํา']
1
Finished batch 1 in 15.231s
seed text: ['<s>', '▁', 'ร่วมด้วย', 'ช่วยกัน']
1
Finished batch 1 in 15.322s
seed text: ['<s>', '▁', 'ถูก', 'ไหม']
1
Finished batch 1 in 10.926s
seed text: ['<s>', '▁ขอ', 'พระองค์ทรง', 'พระ']
1
Finished batch 1 in 11.078s
seed text: ['<s>', '▁', 'เป็น', 'กําลังใจ']
1
Finished batch 1 in 11.149s
seed text: ['<s>', '▁ทหาร', 'เป็น', 'ที่พึ่ง']
1
Finished batch 1 in 10.877s
seed text: ['<s>', '▁', '#', 'ใคร']
1
Finished batch 1 in 10.947s
seed text: ['<s>', '▁', 'ไอ้หนู', 'กลับไป']
1
Finished batch 1 in 11.054s
seed text: ['<s>', '▁คิด', 'ก่อน', 'แชร์']
1
Finished batch 1 in 10.872s
seed text: ['<s>', '▁ก็', 'ดีครับ', '▁']
1
Finished batch 1 in 15.040s
seed text: ['<s>', '▁เด็ก', 'พปชร', '.']
1
Finished batch 1 in 14.790s
seed text: ['<s>', '▁', 'สนับสนุน', '▁']
1
Finished batch 1 in 10.948s
seed text: ['<s>', '▁', 'กําลังพล', 'ตัวอย่าง']
1
Finished batch 1 in 11.174s
seed text: ['<s>', '▁

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Finished batch 1 in 10.901s
seed text: ['<s>', '▁', 'ปาก', 'หมา']
1
Finished batch 1 in 10.994s
seed text: ['<s>', '▁', 'เป็นกําลัง', 'ให้']
1
Finished batch 1 in 10.927s
seed text: ['<s>', '▁มันก็', 'เผือก', 'กก']
1
Finished batch 1 in 11.007s
seed text: ['<s>', '▁', 'ยุบ', 'ๆ']
1
Finished batch 1 in 10.886s
seed text: ['<s>', '▁', 'ติดตาม', 'เลยครับ']
1
Finished batch 1 in 11.086s
seed text: ['<s>', '▁', 'กระทรวงการ', 'ต่างประเทศ']
1
Finished batch 1 in 14.509s
seed text: ['<s>', '▁ทหาร', 'เป็น', 'ที่พึ่ง']
1
Finished batch 1 in 10.855s
seed text: ['<s>', '▁', 'เป็นกําลัง', 'ให้']
1
Finished batch 1 in 11.159s
seed text: ['<s>', '▁', 'จะทําอะไร', 'ก็']
1
Finished batch 1 in 10.993s
seed text: ['<s>', '▁ยังคง', 'ต้องไป', 'ใช้บริการ']
1
Finished batch 1 in 11.105s
seed text: ['<s>', '▁', 'don', 't']
1
Finished batch 1 in 11.004s
seed text: ['<s>', '▁ไม่', 'ครับ', '▁ไม่ต้อง']
1
Finished batch 1 in 14.517s
seed text: ['<s>', '▁', 'ออกมา', 'ข้างนอก']
1
Finished batch 1 in 10.993s
seed tex

In [ ]:
len(refs2)

400

Final score of 500 sentence 

In [ ]:
# 500 sentences score
corpus_bleu(refs+refs2, cans+cans2)

0.3105449257560133